#Label a big set of images can be expensive, time consuming and tedious, so that the data augmentation appears as a solution to this problem, but we have to be careful with the over fitting

In [ ]:
def load_data(FILEID):
    #FILEID es la identificación de un archivo .zip en mi drive con permisos para cualquiera que posea el ID
    ## dicho ID se encuentra en celdas más abajo.
    #Se carga, descomprime y finalmente guarda en la variable Data el conjunto de datos.
    !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O parches.zip && rm -rf /tmp/cookies.txt
    !unzip parches.zip
    !dir

    #Data=df.read_csv('/content/UpdatedResumeDataSet.csv')

In [ ]:
load_data('19GMUbmLY8av-Z4yU3DpkozexGXbAgNnM')

--2025-02-07 14:15:24--  https://docs.google.com/uc?export=download&confirm=&id=19GMUbmLY8av-Z4yU3DpkozexGXbAgNnM
Resolving docs.google.com (docs.google.com)... 172.253.118.101, 172.253.118.113, 172.253.118.138, ...
Connecting to docs.google.com (docs.google.com)|172.253.118.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=19GMUbmLY8av-Z4yU3DpkozexGXbAgNnM&export=download [following]
--2025-02-07 14:15:25--  https://drive.usercontent.google.com/download?id=19GMUbmLY8av-Z4yU3DpkozexGXbAgNnM&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.175.132, 2404:6800:4003:c1c::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.175.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2422 (2.4K) [text/html]
Saving to: ‘parches.zip’

parches.zip         100%[===================>]   2.37K  --.-KB/s    in 0s  

In [ ]:
#Packages:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img #For do Data Augmentation
import matplotlib.pyplot as plt

In [ ]:
#Define the objetc for do DataAugmentation (Every parameter description is in the documentation)
datage=ImageDataGenerator(rotation_range=40,zoom_range=0.2,horizontal_flip=True,fill_mode="nearest",brightness_range=[0.4,1.5])

In [ ]:
img=load_img("/content/Mara.jpeg") #Loading the image
img

FileNotFoundError: [Errno 2] No such file or directory: '/content/Mara.jpeg'

In [ ]:
x=img_to_array(img) #Convert the image to an array
x.shape

In [ ]:
x.shape=(1,800,600,3) #We only have 1 image

In [ ]:
x.shape

In [ ]:
#Running our image generator for do the data augmentation:
i=0
for batch in datage.flow(x,batch_size=1):
  plt.figure(i)
  imgplot=plt.imshow(array_to_img(batch[0]))
  i+=1
  if i%10==0: #For stop the generation of image
    break
  plt.show()

In [ ]:
#Genering Data Augmentation over all image of the train directoty:
train_generator=datage.flow_from_directory("/content/cats_and_dogs/train",target_size=(150,150),batch_size=32,class_mode="binary") #Resolution (150x150), binary because in the image of train directory I only have 2 classes

#That show me what found in the directory

In [ ]:
train_generator[0][0].shape #Images

In [ ]:
train_generator[0][1].shape #Labels

In [ ]:
len(train_generator)*32 #Now I have  2016 images (for the data aumentation) more the 2000 original images

In [ ]:
#Plotting a image:
array_to_img(train_generator[0][0][5])

In [ ]:
from keras.api._v2.keras import regularizers
#Packages:
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.datasets import cifar10

import numpy as np
import matplotlib.pyplot as plt
base_filtros=32 #Number of Kernels in the convolutional layers
w_regularizer=1e-4 #weight of the relularizer

model=Sequential()

#First convolution layer:
model.add(Conv2D(base_filtros,(3,3),padding="same",kernel_regularizer=regularizers.l2(w_regularizer), input_shape=(150,150,3))) #put a regularizer with l2
model.add(Activation("relu")) #Put a Activation layer of type relu

#Second convolution layer:
model.add(Conv2D(base_filtros,(3,3),padding="same",kernel_regularizer=regularizers.l2(w_regularizer))) #put a regularizer with l2
model.add(Activation("relu")) #Put a Activation layer of type relu
model.add(MaxPooling2D(pool_size=(2,2))) #First MaxPooling Layer
model.add(Dropout(0.2))


#Convolution layer number 3 (Iqual but increase the number of kernels):
model.add(Conv2D(2*base_filtros,(3,3),padding="same",kernel_regularizer=regularizers.l2(w_regularizer))) #put a regularizer with l2
model.add(Activation("relu")) #Put a Activation layer of type relu
model.add(Dropout(0.2))


#Convolution layer number 4:
model.add(Conv2D(2*base_filtros,(3,3),padding="same",kernel_regularizer=regularizers.l2(w_regularizer))) #put a regularizer with l2
model.add(Activation("relu")) #Put a Activation layer of type relu
model.add(MaxPooling2D(pool_size=(2,2))) #Second MaxPooling Layer
model.add(Dropout(0.3))



#Convolution layer number 5 (increase the number of kernels):
model.add(Conv2D(4*base_filtros,(3,3),padding="same",kernel_regularizer=regularizers.l2(w_regularizer))) #put a regularizer with l2
model.add(Activation("relu")) #Put a Activation layer of type relu

#Convolution layer number 6:
model.add(Conv2D(4*base_filtros,(3,3),padding="same",kernel_regularizer=regularizers.l2(w_regularizer))) #put a regularizer with l2
model.add(Activation("relu")) #Put a Activation layer of type relu
model.add(MaxPooling2D(pool_size=(2,2))) #MaxPooling Layer Number three
model.add(Dropout(0.4))

#Classifiation (with dense layer) and Flatten:
model.add(Flatten())
model.add(Dense(1,activation="sigmoid"))


In [ ]:
#Compile the model:
model.compile(metrics=["accuracy"],loss="binary_crossentropy",optimizer="rmsprop")

In [ ]:
#Training the model:
history=model.fit(train_generator,epochs=5)

In [ ]:
#Genering Data Augmentation over all image of the train directoty:
test_generator=datage.flow_from_directory("/content/cats_and_dogs/test",target_size=(150,150),batch_size=32,class_mode="binary",) #Resolution (150x150), binary because in the image of train directory I only have 2 classes

#That show me what found in the directory

In [ ]:
model.evaluate(test_generator)

In [ ]:
val_generator=datage.flow_from_directory("/content/cats_and_dogs/validation",target_size=(150,150),batch_size=32,class_mode="binary",) #Resolution (150x150), binary because in the image of train directory I only have 2 classes

#That show me what found in the directory

In [ ]:
model.fit(train_generator,epochs=5,validation_data=val_generator)